# GPT batch script

This routine shows an example of how to use python to call SNAP's GPT processor, to allow automated processing of S3A OLCI L1 images from the command line.

    Version: 1.0
    Author: C Lebreton, Brockmann Consult
    Notes:
    1. A batch scripting version of the same file, batch_gpt.py, can be found in the same folder.
    2. Both this script, and the batch version, will require you to adapt the paths to the GPT executable and data
       on your local system.
      

We begin by importing some python modules

In [1]:
import os
from os import path, listdir, system

Next we have to tell our code where the GPT executable is. The default path is hown below, but you may have to adapt this depending on where you installed SNAP.

In [2]:
# CONFIG
# set up the environment, path to your gpt processor
gptProcessor = '/usr/local/snap6/bin/gpt'

Now we must point the code to our config file (in the same directory as this file), and the data we downloaded. It is the configuration file that describes the processes that SNAP will perform in our automated chain.

In [3]:
# define your graph xml, and input and output directory paths
c2rcc_xml = path.join(os.getcwd(),'olci_c2rcc.xml')
input_dir = path.join(os.getcwd(),'L1_subset')
output_dir = path.join(os.getcwd(),'L2_batch')

Now we search the input directory to find Level-1 Sentinel-3 data

In [4]:
# INPUTS
# make a list of all the input files in your input directory, only taking the .dim files
input_files = [f for f in sorted(listdir(input_dir)) if
               path.isfile(path.join(input_dir, f)) and path.basename(f).endswith('.dim')]

And finally, we start the main part of our code, where we call GPT to process this L1 data. 

In [5]:
# MAIN
# this is the loop where the magic happens :-)
# the loop goes over each input file in the input_files list
for input_file in input_files:
    # for gpt, you need the full path to the file (or you need to set your working environment)
    input = path.join(input_dir, input_file)
    print(input)
    # the output file is named from the input file with the _C2RCCC suffix.
    # Here you will write a netcdf file. this can be changed
    output_product = path.join(output_dir, path.splitext(input_file)[0] + '_C2RCC.nc')
    print(output_product)
    # the processing call is a follows below. Make sure that you have the necessary spaces in between the calls
    c2rcc_processingCall = gptProcessor + ' ' + c2rcc_xml + ' -SsourceProduct=' + input + ' -t ' + output_product
    # useful to check that the command call is correct before launching the call (comment / uncomment the next line)
    print(c2rcc_processingCall)
    # python call, uncomment when the printed call satisfies your requirements
    system(c2rcc_processingCall)

/shared/GPT/L1_subset/S3A_OL_1_EFR____20170527T100953_20170527T101253_20170528T142840_0179_018_122_1980_LN1_O_NT_002_subset.dim
/shared/GPT/L2_batch/S3A_OL_1_EFR____20170527T100953_20170527T101253_20170528T142840_0179_018_122_1980_LN1_O_NT_002_subset_C2RCC.nc
/usr/local/snap6/bin/gpt /shared/GPT/olci_c2rcc.xml -SsourceProduct=/shared/GPT/L1_subset/S3A_OL_1_EFR____20170527T100953_20170527T101253_20170528T142840_0179_018_122_1980_LN1_O_NT_002_subset.dim -t /shared/GPT/L2_batch/S3A_OL_1_EFR____20170527T100953_20170527T101253_20170528T142840_0179_018_122_1980_LN1_O_NT_002_subset_C2RCC.nc
/shared/GPT/L1_subset/S3A_OL_1_EFR____20170528T094342_20170528T094642_20170528T114414_0179_018_136_1979_SVL_O_NR_002_subset.dim
/shared/GPT/L2_batch/S3A_OL_1_EFR____20170528T094342_20170528T094642_20170528T114414_0179_018_136_1979_SVL_O_NR_002_subset_C2RCC.nc
/usr/local/snap6/bin/gpt /shared/GPT/olci_c2rcc.xml -SsourceProduct=/shared/GPT/L1_subset/S3A_OL_1_EFR____20170528T094342_20170528T094642_20170528T11